<a href="https://colab.research.google.com/github/AbakirH/CS301_Project_Group7/blob/main/CS301_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group Members: Abakir Hanna, Edel Barcenas, Gabriela Banayotti

Abstract: Briefly describe your problem, approach, and key results. Should be no more than 300 words.



Introduction (10%): Describe the problem you are working on, why it’s important, and an overview of your results



Related Work (10%): Discuss published work that relates to your project. How is your approach similar or different from others?



Data (10%): Describe the data you are working with for your project. What type of data is it? Where did it come from? How much data are you working with? Did you have to do any preprocessing, filtering, or other special treatment to use this data in your project?



Methods (30%): Discuss your approach for solving the problems that you set up in the introduction. Why is your approach the right thing to do? Did you consider alternative approaches? You should demonstrate that you have applied ideas and skills built up during the quarter to tackling your problem of choice. It may be helpful to include figures, diagrams, or tables to describe your method or compare it with other methods.



Experiments (30%): Discuss the experiments that you performed to demonstrate that your approach solves the problem. The exact experiments will vary depending on the project, but you might compare with previously published methods, perform an ablation study to determine the impact of various components of your system, experiment with different hyperparameters or architectural choices, use visualization techniques to gain insight into how your model works, discuss common failure modes of your model, etc. You should include graphs, tables, or other figures to illustrate your experimental results.


Conclusion (5%) Summarize your key results - what have you learned? Suggest ideas for future extensions or new applications of your ideas.

#Intoduction

The NCAA March Madness is an annual, single elimination, American, College Basketball tournament. It consists of a 64 team bracket. The goal of this paper is to accurately predict the winner of any game within the tournament, and potentially(although unlikely) the entire bracket. Why bother predicting the outcome of a basketball tournament? With 10’s of millions of viewers, 18 million just for the final game, there is substantial financial insensitivity in the betting realm. An accurate \$1 bet that predicts the entire bracket for the 32 teams left after the first round pays out $15,000,000. A semi-accurate model can be very lucrative. Using a plethora of data described below, this model was able to make predictions with a Local Cross Validation Score of 0.659.

#Related Work

#Data

The data being used comes form /content/kaggle/mens-march-mania-2022. This includes results from March Madness Events dating back to 2016. However, win/loss ratios, win/loss score differentials, and seeding are also available. Also, team information from the regular season can be implemented into the model's training. 

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import keras
keras.__version__

'2.8.0'

In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abakirhanna","key":"7b47ec063ed3d688514c9878a5c78b24"}'}

In [ ]:
!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.12


In [ ]:
%ls /root/.kaggle/

kaggle.json


In [ ]:
!kaggle competitions files -c mens-march-mania-2022

name                                                    size  creationDate         
-----------------------------------------------------  -----  -------------------  
MDataFiles_Stage2/MSeasons.csv                           2KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MSecondaryTourneyTeams.csv            24KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv        14KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MNCAATourneyCompactResults.csv        68KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MNCAATourneySeeds.csv                 36KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MGameCities.csv                        2MB  2022-03-14 21:29:06  
MDataFiles_Stage2/MTeams.csv                            10KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MRegularSeasonCompactResults.csv       5MB  2022-03-14 21:29:06  
MDataFiles_Stage2/Cities.csv                             9KB  2022-03-14 21:29:06  
MDataFiles_Stage2/MTeamConferences.csv                 207KB  2022-03-14 21:

In [ ]:
!kaggle competitions download  -c mens-march-mania-2022 -p /content/kaggle/mens-march-mania-2022 

 72% 33.0M/45.8M [00:00<00:00, 121MB/s] 
100% 45.8M/45.8M [00:00<00:00, 139MB/s]


In [ ]:
import os, shutil

!ls /content/kaggle/mens-march-mania-2022


mens-march-mania-2022.zip


In [ ]:
!unzip -q /content/kaggle/mens-march-mania-2022/mens-march-mania-2022.zip -d /content/kaggle/mens-march-mania-2022


In [ ]:
import jax.numpy as jnp
import pandas as pd 
import re 

In [ ]:
# Read the requiered datasets.
!ls /content/kaggle/mens-march-mania-2022

MDataFiles_Stage1  MDataFiles_Stage2  mens-march-mania-2022.zip


In [ ]:
season_data = pd.read_csv('/content/kaggle/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
seeds = pd.read_csv('/content/kaggle/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeeds.csv')

public_rating = pd.read_csv('/content/kaggle/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')

In [ ]:
# Add a new column to know the differnce of points between those who in and lose
def score_gap(data):
    data['ScoreGapBetweenTeamsPlayed'] = data['WScore'] - data['LScore']
    return data
  
season_data = score_gap(season_data)

In [ ]:
season_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ScoreGapBetweenTeamsPlayed
0,1985,20,1228,81,1328,64,N,0,17
1,1985,25,1106,77,1354,70,H,0,7
2,1985,25,1112,63,1223,56,H,0,7
3,1985,25,1165,70,1432,54,H,0,16
4,1985,25,1192,86,1447,74,H,0,12


In [ ]:
def create_team_list(data, group_list = ['Season', 'WTeamID'], team_id = 'WTeamID'):
    group = data.groupby(group_list).count().reset_index()
    #We only care about the teams that were played in the last 6 years as rules have changes over time
    group = data.loc[(data['Season'] > 2015)]
    group = group[group_list].rename(columns={team_id: "TeamID"})
    return group

winners = create_team_list(season_data, group_list = ['Season', 'WTeamID'], team_id = 'WTeamID')
lossers = create_team_list(season_data, group_list = ['Season', 'LTeamID'], team_id = 'LTeamID')

# Create an empty train dataset that we will fill in ourselvs.
team_agg_features = pd.concat([winners, lossers], axis = 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop = True)
team_agg_features

,Season,TeamID
0,2016,1101
1,2016,1102
2,2016,1103
3,2016,1104
4,2016,1105
...,...,...
2459,2022,1468
2460,2022,1469
2461,2022,1470
2462,2022,1471


#Methods & Experiment

In [ ]:
def winner_aggregated_features(df, group_list = ['Season', 'WTeamID']):
    tmp = df.groupby(group_list).agg(NumWins       = ('WTeamID', 'count'), 
                                     AvgWinsGap    = ('ScoreGapBetweenTeamsPlayed', 'mean'),
                                     W_TotalPoints = ('WScore', 'sum'),
                                     W_MaxPoints   = ('WScore', 'max'),
                                     W_MinPoints   = ('WScore', 'min'),
                                    )
    tmp = tmp.reset_index()
    tmp = tmp.rename(columns={"WTeamID": "TeamID"})
    
    return tmp

def losser_aggregated_features(df, group_list = ['Season', 'LTeamID']):
    tmp = df.groupby(group_list).agg(NumLosses       = ('LTeamID', 'count'), 
                                     AvgLossesGap    = ('ScoreGapBetweenTeamsPlayed', 'mean'),
                                     L_TotalPoints = ('LScore', 'sum'),
                                     L_MaxPoints   = ('LScore', 'max'),
                                     L_MinPoints   = ('LScore', 'min'),
                                    )
    tmp = tmp.reset_index()
    tmp = tmp.rename(columns={"LTeamID": "TeamID"})
    return tmp
  
winner_team_aggregation = winner_aggregated_features(season_data)
losser_team_aggregation = losser_aggregated_features(season_data)


def merge_back(df):
    df = df.merge(winner_team_aggregation, on = ['Season', 'TeamID'], how = 'left')
    df = df.merge(losser_team_aggregation, on = ['Season', 'TeamID'], how = 'left')
    df.fillna(0, inplace = True) 
    return df

team_agg_features = merge_back(team_agg_features)
team_agg_features

,Season,TeamID,WinRatio,AvgScoreGap,PointsRatio,NumWins,AvgWinsGap,W_TotalPoints,W_MaxPoints,W_MinPoints,NumLosses,AvgLossesGap,L_TotalPoints,L_MaxPoints,L_MinPoints
0,2016,1101,0.333333,-6.407407,0.373277,9.0,7.555556,704.0,91.0,62.0,18.0,13.388889,1182.0,100.0,51.0
1,2016,1102,0.400000,-6.166667,0.420168,12.0,5.833333,850.0,97.0,61.0,18.0,14.166667,1173.0,102.0,50.0
2,2016,1103,0.757576,7.272727,0.784041,25.0,12.520000,1975.0,92.0,62.0,8.0,9.125000,544.0,81.0,56.0
3,2016,1104,0.562500,-0.968750,0.598410,18.0,7.944444,1280.0,105.0,51.0,14.0,12.428571,859.0,77.0,45.0
4,2016,1105,0.357143,-2.642857,0.386908,10.0,8.500000,727.0,85.0,54.0,18.0,8.833333,1152.0,78.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2022,1468,0.450000,-4.300000,0.503754,9.0,10.666667,671.0,85.0,66.0,11.0,16.545455,661.0,70.0,50.0
2460,2022,1469,0.368421,-8.000000,0.421650,7.0,11.428571,557.0,85.0,71.0,12.0,19.333333,764.0,73.0,50.0
2461,2022,1470,0.380952,-3.047619,0.424174,8.0,11.125000,565.0,83.0,57.0,13.0,11.769231,767.0,84.0,50.0
2462,2022,1471,0.400000,-4.000000,0.470939,8.0,10.125000,632.0,85.0,71.0,12.0,13.416667,710.0,80.0,48.0


In [ ]:
def calculate_features(df):

    df['WinRatio'] = df['NumWins'] / (df['NumWins'] + df['NumLosses'])
    df['AvgScoreGap'] = ((df['NumWins'] * df['AvgWinsGap'] - df['NumLosses'] * df['AvgLossesGap']) / (df['NumWins'] + df['NumLosses']))
    df['PointsRatio'] = df['W_TotalPoints'] / (df['L_TotalPoints'] + df['W_TotalPoints'])
    return df

team_agg_features = calculate_features(team_agg_features)
team_agg_features = team_agg_features[['Season','TeamID','WinRatio', 'AvgScoreGap','PointsRatio']]

In [ ]:
tournament_data = pd.read_csv('/content/kaggle/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tournament_data = tournament_data.rename(columns = {'WTeamID' : 'W_TeamID', 'LTeamID' : 'L_TeamID', 'WScore' : 'W_Score', 'LScore' : 'L_Score'})
tournament_data.drop(['NumOT', 'WLoc'], axis = 1, inplace = True)
tournament_data = tournament_data[tournament_data['Season'] > 2015].reset_index(drop = True)
tournament_data.head()


,Season,DayNum,W_TeamID,W_Score,L_TeamID,L_Score
0,2016,134,1195,96,1192,65
1,2016,134,1455,70,1435,50
2,2016,135,1221,59,1380,55
3,2016,135,1276,67,1409,62
4,2016,136,1114,85,1345,83


In [ ]:
def merge_seed(df, seed_df, left_on = ['Season', 'W_TeamID'], field_name = 'SeedW'):
    df = pd.merge(df,seed_df, how = 'left', left_on = left_on, right_on = ['Season', 'TeamID'])
    df = df.drop('TeamID', axis = 1).rename(columns = {'Seed': field_name})
    return df
tournament_data = merge_seed(tournament_data, seeds, left_on = ['Season', 'W_TeamID'], field_name = 'W_Seed')
tournament_data = merge_seed(tournament_data, seeds, left_on = ['Season', 'L_TeamID'], field_name = 'L_Seed')

In [ ]:
def seed_number(row):
    return int(re.sub("[^0-9]", "", row))

tournament_data['W_Seed'] = tournament_data['W_Seed'].apply(seed_number)
tournament_data['L_Seed'] = tournament_data['L_Seed'].apply(seed_number)

In [ ]:
def merge_agg_features(df, agg_features):
    for result in ['W', 'L']:
        df = pd.merge(df, agg_features, how = 'left', left_on = ['Season', result +'_'+ 'TeamID'], right_on = ['Season', 'TeamID'])
        avoid = ['Season', 'TeamID']
        new_names = {col: result +'_'+ col for col in agg_features.columns if col not in avoid}
        df = df.rename(columns = new_names)        
        df = df.drop(columns = 'TeamID', axis = 1)
    return df

tournament_data = merge_agg_features(tournament_data, team_agg_features)

In [ ]:
def replace_win_loser(df):
    team_a = df.copy()
    team_b = df.copy()
    
    team_a_dict, team_b_dict = {}, {}
    
    for col in team_a.columns:
        if col.find('W_') == 0:
            new_col_name = str(col).replace('W_', 'A_')
            team_a_dict[col] = new_col_name
        if col.find('L_') == 0:
            new_col_name = col.replace('L_', 'B_')    
            team_a_dict[col] = new_col_name
            
    for col in team_b.columns:
        if col.find('W_') == 0:
            new_col_name = str(col).replace('W_', 'B_')
            team_b_dict[col] = new_col_name
        if col.find('L_') == 0:
            new_col_name = col.replace('L_', 'A_')
            team_b_dict[col] = new_col_name

    team_a = team_a.rename(columns = team_a_dict)
    team_b = team_b.rename(columns = team_b_dict)
    
    merged_df = pd.concat([team_a, team_b], axis = 0, sort = False)
    return merged_df

In [ ]:
tournament_data = replace_win_loser(tournament_data)

In [ ]:
tournament_data

,Season,DayNum,A_TeamID,A_Score,B_TeamID,B_Score,A_Seed,B_Seed,A_WinRatio,A_AvgScoreGap,A_PointsRatio,B_WinRatio,B_AvgScoreGap,B_PointsRatio
0,2016,134,1195,96,1192,65,16,16,0.551724,2.724138,0.591697,0.548387,-1.677419,0.589001
1,2016,134,1455,70,1435,50,11,11,0.741935,13.709677,0.784738,0.593750,9.406250,0.640472
2,2016,135,1221,59,1380,55,16,16,0.424242,-4.333333,0.458256,0.612903,1.870968,0.641304
3,2016,135,1276,67,1409,62,11,11,0.636364,6.242424,0.676710,0.645161,4.322581,0.665650
4,2016,136,1114,85,1345,83,12,5,0.870968,9.935484,0.875115,0.764706,13.147059,0.801740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2021,148,1425,66,1211,85,6,1,0.758621,9.655172,0.796587,1.000000,23.000000,1.000000
330,2021,148,1276,49,1417,51,1,11,0.833333,10.875000,0.868306,0.653846,4.346154,0.684266
331,2021,152,1222,59,1124,78,2,1,0.884615,18.000000,0.899194,0.916667,17.958333,0.934815
332,2021,152,1417,90,1211,93,11,1,0.653846,4.346154,0.684266,1.000000,23.000000,1.000000


In [ ]:
def calculate_differences(df):
    df['SeedDiff'] = df['A_Seed'] - df['B_Seed']
    df['WinRatioDiff'] = df['A_WinRatio'] - df['B_WinRatio']
    df['GapAvgDiff'] = df['A_AvgScoreGap'] - df['B_AvgScoreGap']    
    df['PointsRatioDiff'] = df['A_PointsRatio'] - df['A_PointsRatio']
    return df

tournament_data = calculate_differences(tournament_data)

In [ ]:
tournament_data['ScoreDiff'] = tournament_data['A_Score'] - tournament_data['B_Score']
tournament_data['A_Win'] = (tournament_data['ScoreDiff'] > 0).astype(int)
tournament_data = tournament_data.drop(columns=['A_Score', 'B_Score'])
tournament_data.head()

,Season,DayNum,A_TeamID,B_TeamID,A_Seed,B_Seed,A_WinRatio,A_AvgScoreGap,A_PointsRatio,B_WinRatio,B_AvgScoreGap,B_PointsRatio,SeedDiff,WinRatioDiff,GapAvgDiff,PointsRatioDiff,ScoreDiff,A_Win
0,2016,134,1195,1192,16,16,0.551724,2.724138,0.591697,0.548387,-1.677419,0.589001,0,0.003337,4.401557,0.0,31,1
1,2016,134,1455,1435,11,11,0.741935,13.709677,0.784738,0.593750,9.406250,0.640472,0,0.148185,4.303427,0.0,20,1
2,2016,135,1221,1380,16,16,0.424242,-4.333333,0.458256,0.612903,1.870968,0.641304,0,-0.188661,-6.204301,0.0,4,1
3,2016,135,1276,1409,11,11,0.636364,6.242424,0.676710,0.645161,4.322581,0.665650,0,-0.008798,1.919844,0.0,5,1
4,2016,136,1114,1345,12,5,0.870968,9.935484,0.875115,0.764706,13.147059,0.801740,7,0.106262,-3.211575,0.0,2,1


In [ ]:
trainingData = pd.read_csv('/content/kaggle/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv')
tst_data = trainingData.copy()

def separate_id(df):
    df['Season']  = df['ID'].apply(lambda x: int(x.split('_')[0]))
    df['TeamIdA'] = df['ID'].apply(lambda x: int(x.split('_')[1]))
    df['TeamIdB'] = df['ID'].apply(lambda x: int(x.split('_')[2]))
    return df

tst_data = separate_id(tst_data)

In [ ]:
tst_data = merge_seed(tst_data, seeds, left_on = ['Season', 'TeamIdA'], field_name = 'A_Seed')
tst_data = merge_seed(tst_data, seeds, left_on = ['Season', 'TeamIdB'], field_name = 'B_Seed')

In [ ]:
tst_data

,ID,Pred,Season,TeamIdA,TeamIdB,A_Seed,B_Seed
0,2016_1112_1114,0.5,2016,1112,1114,Y06,X12
1,2016_1112_1122,0.5,2016,1112,1122,Y06,Y16
2,2016_1112_1124,0.5,2016,1112,1124,Y06,Z05
3,2016_1112_1138,0.5,2016,1112,1138,Y06,Y14
4,2016_1112_1139,0.5,2016,1112,1139,Y06,X09
...,...,...,...,...,...,...,...
11385,2021_1452_1457,0.5,2021,1452,1457,Y03,Z12
11386,2021_1452_1458,0.5,2021,1452,1458,Y03,Z09
11387,2021_1455_1457,0.5,2021,1455,1457,X11b,Z12
11388,2021_1455_1458,0.5,2021,1455,1458,X11b,Z09


In [ ]:
tst_data['A_Seed'] = tst_data['A_Seed'].apply(seed_number)
tst_data['B_Seed'] = tst_data['B_Seed'].apply(seed_number)
tst_data = tst_data.rename(columns = {'TeamIdA': 'A_TeamID', 'TeamIdB': 'B_TeamID'})

In [ ]:
team_agg_features

,Season,TeamID,WinRatio,AvgScoreGap,PointsRatio
0,2016,1101,0.333333,-6.407407,0.373277
1,2016,1102,0.400000,-6.166667,0.420168
2,2016,1103,0.757576,7.272727,0.784041
3,2016,1104,0.562500,-0.968750,0.598410
4,2016,1105,0.357143,-2.642857,0.386908
...,...,...,...,...,...
2459,2022,1468,0.450000,-4.300000,0.503754
2460,2022,1469,0.368421,-8.000000,0.421650
2461,2022,1470,0.380952,-3.047619,0.424174
2462,2022,1471,0.400000,-4.000000,0.470939


In [ ]:
def merge_agg_features(df, agg_features):
    for result in ['A', 'B']:
        df = pd.merge(df, agg_features, how = 'left', left_on = ['Season', result +'_'+ 'TeamID'], right_on = ['Season', 'TeamID'])
        avoid = ['Season', 'TeamID']
        new_names = {col: result +'_'+ col for col in agg_features.columns if col not in avoid}
        df = df.rename(columns = new_names)        
        df = df.drop(columns = 'TeamID', axis = 1)
    return df

tst_data = merge_agg_features(tst_data, team_agg_features)

In [ ]:
tst_data = calculate_differences(tst_data)

In [ ]:
from sklearn import tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier

target_feature = 'A_Win'
avoid = ['ScoreDiff', 'Season', 'DayNum', 'A_Win']
features = [col for col in tournament_data.columns if col not in avoid]
for feature in features:
  print(feature)

A_TeamID
B_TeamID
A_Seed
B_Seed
A_WinRatio
A_AvgScoreGap
A_PointsRatio
B_WinRatio
B_AvgScoreGap
B_PointsRatio
SeedDiff
WinRatioDiff
GapAvgDiff
PointsRatioDiff


In [ ]:
def log_loss_function(y_val, y_pred):
    logLossSum = 0
    for x in range(0,len(y_val)):
        logLossSum += (y_val[x]*jnp.log(y_pred[x]) + (1-y_val[x])*jnp.log(1-y_pred[x]))
        
    logLossSum = -logLossSum/len(y_val)
    return logLossSum

In [ ]:
def kfold_model(train_df, tst_df):
    cvs = []
    preds_test = []
    seasons = train_df['Season'].unique()
    
    for season in seasons[1:]:
        print(f'\nValidating on season {season}')
        X_train = train_df[train_df['Season'] < season][features].reset_index(drop = True).copy()
        X_val = train_df[train_df['Season'] == season][features].reset_index(drop = True).copy()
        
        y_train = train_df[train_df['Season'] < season][target_feature].reset_index(drop = True).copy()
        y_val = train_df[train_df['Season'] == season][target_feature].reset_index(drop = True).copy()
        
        tst_dataset = tst_data[features].copy()
        
        
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        
        X_train = scaler.transform(X_train)        
        X_val = scaler.transform(X_val)
        tst_dataset = scaler.transform(tst_dataset)
        
        model = XGBClassifier(n_estimators = 1024, random_state = 85)
        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 0, early_stopping_rounds = 128)
        
        pred_test = model.predict_proba(tst_dataset)[:, 1]
        preds_test.append(pred_test)
        
        print(len(y_val),len(pred_test))
        loss = log_loss_function(y_val, pred_test)

        cvs.append(loss)
        
        print(f'\t -> Scored {loss:.4f}')
    print(f'\nLocal Cross Validation Score Is: {jnp.mean(jnp.array(cvs)):.3f}', '\n')
    return preds_test

In [ ]:
predictions = kfold_model(tournament_data, tournament_data)


Validating on season 2017
134 11390
	 -> Scored 0.5900

Validating on season 2018
134 11390
	 -> Scored 0.6101

Validating on season 2019
134 11390
	 -> Scored 0.8386

Validating on season 2021
132 11390
	 -> Scored 0.5966

Local Cross Validation Score Is: 0.659 



#Conclusion